In [1]:
import aplusml
import numpy as np
from typing import List, Dict
import collections

############################################################
# 1. Initialize simulation
############################################################

# Create Config object. You must specify three things:
# 1. Metadata -- basic information about the simulation
# 2. Variables -- variables used in the simulation
# 3. States -- states in the workflow that your patients will progress through
config = aplusml.config.Config(
  metadata = aplusml.config.ConfigMetadata(
    name = 'My Simulation',
  ),
  variables = {
    'some_constant': aplusml.config.ConfigVariable(
      type = 'scalar',
      value = -12.03,
    ),
    'some_resource': aplusml.config.ConfigVariable(
      type = 'resource',
      init_amount = 0,
      max_amount = 10,
      refill_amount = 3,
      refill_duration = 1,
    ),
  },
  states = {
    'start' : aplusml.config.ConfigState(
      type = 'start',
      transitions = [
        aplusml.config.ConfigTransition(dest = 'end_1', prob = 0.5),
        aplusml.config.ConfigTransition(dest = 'end_2', prob = 0.5),
      ],
    ),
    'end_1' : aplusml.config.ConfigState(
        type = 'end',
        utilities = [
            aplusml.config.ConfigUtility(
                value = .5,
                unit = 'qaly',
            ),
        ],
    ),
    'end_2' : aplusml.config.ConfigState(
        type = 'end',
        utilities = [
            aplusml.config.ConfigUtility(
                value = 2,
                unit = 'qaly',
            ),
        ],
    ),
  },
)

# Create Simulation object
simulation: aplusml.Simulation = aplusml.Simulation.create_from_config(config)

# Set random seed for reproducibility
np.random.seed(0)

In [2]:
simulation

self.metadata:
    - name: My Simulation
    - path_to_properties: None
    - properties_col_for_patient_id: None
    - patient_sort_preference_property: None

self.variables:
    - some_constant: {'type': 'scalar', 'value': -12.03}
    - some_resource: {'type': 'resource', 'value': None, 'init_amount': 0, 'max_amount': 10, 'refill_amount': 3, 'refill_duration': 1}

self.states:
    - start
        - end_1
            prob: 0.5
        - end_2
            prob: 0.5
    - end_1
    - end_2

In [3]:
############################################################
# 2. Initialize patients
############################################################

# Simulate number of patients per day
n_admits_per_day = np.random.poisson(lam=35, size=500)

# Create empty Patient objects (with proper start timesteps according to our Poisson distribution)
patients: List[aplusml.Patient] = []
for timestep, n_admits in enumerate(n_admits_per_day):
    for x in range(n_admits):
        patients.append(aplusml.Patient(
            len(patients), # Unique ID
            timestep, # Start timestep
        ))

# Function which matches a patient to a row in the CSV file.
func_match_patient_to_property_column = lambda p_id, random_idx, df, col: df.iloc[random_idx][col]

# Initialize patients for simulation.
# This creates a deep copy of each object in the `patients` array using pickle, sorts them by ID, and then initializes their properties
patients: List[aplusml.Patient] = simulation.create_patients_for_simulation(patients,
                                                                          func_match_patient_to_property_column,
                                                                          random_seed = 0)


!! WARNING - Because `is_overwrite_existing_properties` is TRUE, we are OVERWRITING each patient's existing properties (i.e. patient.properties) with default values from `simulation.variables`. If this is UNDESIRED, i.e. you already defined each patient's properties manually and want to keep them as currently defined, then set `is_overwrite_existing_properties` to FALSE.



In [5]:
patients

[Patient({'id': 0, 'start_timestep': 0, 'properties': {}, 'history': [], 'current_state': None}),
 Patient({'id': 1, 'start_timestep': 0, 'properties': {}, 'history': [], 'current_state': None}),
 Patient({'id': 2, 'start_timestep': 0, 'properties': {}, 'history': [], 'current_state': None}),
 Patient({'id': 3, 'start_timestep': 0, 'properties': {}, 'history': [], 'current_state': None}),
 Patient({'id': 4, 'start_timestep': 0, 'properties': {}, 'history': [], 'current_state': None}),
 Patient({'id': 5, 'start_timestep': 0, 'properties': {}, 'history': [], 'current_state': None}),
 Patient({'id': 6, 'start_timestep': 0, 'properties': {}, 'history': [], 'current_state': None}),
 Patient({'id': 7, 'start_timestep': 0, 'properties': {}, 'history': [], 'current_state': None}),
 Patient({'id': 8, 'start_timestep': 0, 'properties': {}, 'history': [], 'current_state': None}),
 Patient({'id': 9, 'start_timestep': 0, 'properties': {}, 'history': [], 'current_state': None}),
 Patient({'id': 10, 

In [6]:
############################################################
# 3. Run simulation
############################################################

patients = simulation.run(patients)

Simulating patients: 100%|██████████| 17515/17515 [00:00<00:00, 48369.93it/s]


In [7]:
patients

[Patient({'id': 0, 'start_timestep': 0, 'properties': {}, 'history': [History({'current_timestep': 0, 'state_id': 'start', 'transition_idx': 1, 'state_utility_idxs': [], 'transition_utility_idxs': [], 'state_utility_vals': [], 'transition_utility_vals': []}), History({'current_timestep': 0, 'state_id': 'end_2', 'transition_idx': None, 'state_utility_idxs': [0], 'transition_utility_idxs': [], 'state_utility_vals': [2], 'transition_utility_vals': []})], 'current_state': None}),
 Patient({'id': 1, 'start_timestep': 0, 'properties': {}, 'history': [History({'current_timestep': 0, 'state_id': 'start', 'transition_idx': 1, 'state_utility_idxs': [], 'transition_utility_idxs': [], 'state_utility_vals': [], 'transition_utility_vals': []}), History({'current_timestep': 0, 'state_id': 'end_2', 'transition_idx': None, 'state_utility_idxs': [0], 'transition_utility_idxs': [], 'state_utility_vals': [2], 'transition_utility_vals': []})], 'current_state': None}),
 Patient({'id': 2, 'start_timestep': 0

In [8]:
############################################################
# 4. Analyze results
############################################################

# Sum up the utilities achieved across all patient histories
sum_utilities: Dict[str, float] = collections.defaultdict(float)
for p in patients:
    _u: dict = p.get_sum_utilities(simulation)
    for key, val in _u.items():
        sum_utilities[key] += val

# Print results
print(sum_utilities)

defaultdict(<class 'float'>, {'qaly': 21917.0})
